<a href="https://colab.research.google.com/github/tluxxx/PortfolioExperiments/blob/main/PortfolioDevelopment_byCummRets_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Study of portfolio development


*   portfolio with constant monthly installments
*   portfolio with adjusted monthly installments
*   actively managed portfolio (depending on trend)






# 1. General Preparations

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import data processing libraries
import pandas as pd
import numpy as np
import datetime as dt

# import plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# import data providing libraries
import yfinance as yf

# 2. Downloading Stock Data and Resampling

In [ ]:
start = '1992-01-01'  # longer than analyzed period, to have later all indicators caclulated (no np.nan)
end = '2024-05-31'
prices = yf.download('^GDAXI', start=start, end=end)

prices_d = prices[prices.index >'1992-12-31']

# aggregation of daily data to monthly data
prices_m = prices_d.resample('M').first()['Close']

[*********************100%%**********************]  1 of 1 completed


# 3. Collection of German Inflation Data

In [ ]:
# reading and preparing inflation data (= Development of Cosumer Price Index Germany)
data = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/InflationDataGermany(1993_2024).xlsx')

xx = data['CPI'][0:-7].copy()
z = np.empty(len(xx))
for i in range(len(xx)):
  z[i] = float(xx[i].replace(",","."))
cpi_dev = pd.Series(z/z[0], index = prices_m.index)

In [ ]:
# Bar-Chart showing the development of consumer price Index in Germany
main_title = 'Development of Consumer Price Index (Germany)'
sub_title = f'from   {start}   to   {end},           note: at reference point  {start} the CPI was set to 100%'
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
fig = go.Figure()
fig.add_trace(go.Scatter(x=cpi_dev.index, y=cpi_dev))
fig.update_layout(template='plotly_dark', autosize=False, width=800, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title=' development of CPI')
fig.show()

# 4. Collection of risk free interest rate data

In [ ]:
# reading and preparing interest data from ECB (1999 -2024) and German Bundesbank (1993-1998))

# German Bundesbank discount rates for every day (approximation, all values as of 01-of each month)
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/DiscountRates_GermanBundesbank(1993-1999).xlsx')
data_db['date'] = pd.to_datetime(data_db['date'])

# generation of data frame with relevant data
## monthly data
start_date, end_date = '1993-01-01', '1998-12-31'

disc_x = (data_db[data_db['date']>=start_date]['intrest_rate'] /100).tolist()
disc_r = pd.DataFrame({'date':data_db[data_db['date']>=start_date]['date'].tolist(),
                       'interest':disc_x})
## conversion to daily data (approximation: if interest rate changes during a month, the change will be implemented at first day of following month
dates = pd.date_range(start_date, end_date)
values = np.nan * len(dates)
dates_d = pd.DataFrame({'date': dates,
                        'interest': values})
xx = pd.merge(dates_d, disc_r, how='left', on='date')['interest_y']
xx.ffill(inplace=True)
db_interest_daily = pd.DataFrame({'interest':xx.tolist()},
                                index = dates)
# ECB interest data for every day
data_ecb = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/EZB interest rates (1999-2024).xlsx')

ecb_int = pd.DataFrame({'interest': data_ecb['EZB_int'].tolist()},
                       index=data_ecb['DATE'].tolist())

# combination of DB and ECB datasets, results transfered to a pd.Series()
interest = pd.concat([db_interest_daily, ecb_int])
interest = interest['interest']

In [ ]:
# Bar-Chart showing the development of risk free interest rates in Germany
main_title = 'development of risk free interest rates (Germany)'
sub_title = f'from   {start}   to   {end}, '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
fig = go.Figure()
fig.add_trace(go.Scatter(x=interest.index, y=interest))
fig.update_layout(template='plotly_dark', autosize=False, width=800, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='risk free interest rate')
fig.show()

# 5. simple monthly investments

*   Method 1 --> monthly installment is constant
*   Method 2 --> mothly installment is adjusted according to consumer price index



In [ ]:
def equity(pr_m, invest_m, fixed_fee, variable_fee):
  ''' simple calculation of equity curve including interest and feese
  Args:
    pr_m (Series):        Close-values of the analyzed instrumend (mlonthly resampled data from daily data)
    invest_m (Series):    Values of fresh montly investment (.e. from a monthly savings plan)
    fixed_fee (float):    Fee in EURO per trade
    variable_fee (float): Fee in percentage of trading volume per trade

  Returns:
      equity (Series):    final Equity Curver
      invested (float):   total invested ammount
      fees (float):       total fees

  '''
  # calculation of newly to be purchases stock
  n = (invest_m - fixed_fee) / pr * (1 - variable_fee)
  nb = np.cumsum(n)
  equity = nb * pr
  invest_total = invest_m.sum()
  fees = (fixed_fee + invest_m * variable_fee).sum()

  return equity, invest_total, fees

In [ ]:
# monthly investment into one instrument

fixed_fee = 4.90          # 4.90 EURO per trade
variable_fee = 0.002      # 0,25% of position

# Method 1: constant monthly installments
invest_c = 100
invest_1 = pd.Series([invest_c] * len(prices_m), index=prices_m.index)
equity_1, invested_1, fees_1 = equity(prices_m, invest_1, fixed_fee, variable_fee)

# Method 2: monthly installments, adjusted according to CPI-development
invest_2 = invest_1 * cpi_dev
equity_2, invested_2, fees_2 = equity(prices_m, invest_2, fixed_fee, variable_fee)


# Chart showing the development of equity
main_title = 'development of equity'
sub_title = f'instrument:   DAX    from   {start}   to   {end} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=equity_1.index, y=equity_1, name='constant monthly investment'))
fig.add_trace(go.Scatter(x=equity_2.index, y=equity_2, name='adjusted monthly investment'))
fig.update_layout(template='plotly_dark', autosize=False, width=800, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='risk free interest rate', legend_title='methods')
fig.show()

In [ ]:
# printout results
growth_factor_1 = equity_1.iloc[-1] / invested_1
growth_factor_2 = equity_2.iloc[-1] / invested_2

print(f'Method 1: constant monthly investment   ******')
print(f'invested ammount (total): {invested_1:,.2f} EURO   final equity:  {equity_1.iloc[-1]:,.2f}  EURO')
print(f'growth-factor:            {growth_factor_1:.2f}             fees:          {fees_1:,.2f}  EURO')
print(' ')
print(f'Method 2: monthly investment (adjusted) ******')
print(f'invested ammount (total): {invested_2:,.2f} EURO   final equity:  {equity_2.iloc[-1]:,.2f}  EURO')
print(f'growth-factor:            {growth_factor_2:.2f}             fees:          {fees_2:,.2f}  EURO')

Method 1: constant monthly investment   ******
invested ammount (total): 37,700.00 EURO   final equity:  122,970.52  EURO
growth-factor:            3.26             fees:          1,922.70  EURO
 
Method 2: monthly investment (adjusted) ******
invested ammount (total): 49,127.40 EURO   final equity:  149,244.54  EURO
growth-factor:            3.04             fees:          1,945.55  EURO


# 6. Adding a TrendFilter


*   Trendfilter by short/long EMA
*   if up-trend then invest in intsrument
*   if no uptrend invest in money fund

In [ ]:
# indicator based trading

# caclulation of EMA' and trend detection -cross oever system
period_short, period_long = 50, 200
x = prices.Close.ewm(span=period_short).mean()
shortEMA = x[x.index > '1992-12-31']
x = prices.Close.ewm(span=period_long).mean()
longEMA = x[x.index > '1992-12-31']
position = pd.Series(np.where(shortEMA > longEMA, 1, 0), index=prices_d.index).resample('M').first()
signal = position.diff()

# calclulation of monthly candlesticks
cs_data = pd.DataFrame({'open': prices_d.resample('M').first()['Open'],
                        'high': prices_d.resample('M').max()['High'],
                        'low': prices_d.resample('M').min()['Low'],
                        'close': prices_d.resample('M').last()['Close']},
                       index=prices_m.index)

text_subtitles = [f'DAX  from {start} to {end}', 'Positions', 'Trading Signals']
fig = make_subplots(rows=3, cols=1, subplot_titles = (text_subtitles), row_heights=[0.8, 0.1, 0.1],
                    shared_xaxes=True, vertical_spacing=0.075 )
fig.add_trace(go.Candlestick(x=cs_data.index,
                             open=cs_data['open'],
                             high=cs_data['high'],
                             low=cs_data['low'],
                             close=cs_data['close'],
                             name='candlestick'), row=1, col=1)
fig.add_trace(go.Scatter(x=shortEMA.index, y=shortEMA, name='short EMA', line_color=' lightskyblue'), row=1, col=1)
fig.add_trace(go.Scatter(x=longEMA.index, y=longEMA, name='long EMA', line_color='yellow'), row=1, col=1)
fig.add_trace(go.Scatter(x=position.index, y=position, name='position', fill = 'tozeroy'), row=2, col=1)
fig.add_trace(go.Scatter(x=signal.index, y=signal, name='signal'), row=3, col=1)
rangeslider=False
fig.update_layout(template='plotly_dark', autosize=False, width=1200, height=600, xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
def equity_trend_filter(pr_m, invest_m, positions_m, signal_m, interest_m, fixed_fee, variable_fee):
  ''' simple calculation of equity curve including interest and feese
  Args:
    pr_m (Series):        Close-values of the analyzed instrumend (mlonthly resampled data from daily data)
    invest_m (Series):    Values of fresh montly investment (.e. from a monthly savings plan)
    positions_m (Series): Positions (0 = LONG, 0 = FLAT )
    signal_m (Series):    Signals (1 = go LONG, -1 exit LONG, go Flat)
    interest_m (Series):  Values of the risk-free-interest-rate
    fixed_fee (float):    Fee in EURO per trade
    variable_fee (float): Fee in percentage of trading volume per trade

  Returns:
      result (DataFrame): param2 (str): Description of param2

  '''

  # strip off pandas overhead, conversion to numpy arrays or generation of new numpy arrays
  # operation to increase speed of calculation (and potential numba usage)
  pr = prices_m.to_numpy()
  ecf = invest_m.to_numpy()
  pos = position.to_numpy()
  sig = signal.to_numpy()
  inr = interest.to_numpy()

  # preparing/initializing the other arrays
  nb = np.zeros(len(pr))
  nbc = np.zeros(len(pr))
  val = np.zeros(len(pr))
  cash = np.zeros(len(pr))
  cash[0] = ecf[0]
  intr_calc = np.zeros(len(pr))
  fee_calc = np.zeros(len(pr))

  # looping through the timeline and checking position/signal, executing dedicated actions
  for i in range(1, len(pr)):

    if (sig[i] == 0) & (pos[i] == 1):          # do nothing, stay LONG: let postition develop, add monthly invest (-fee) to portfolio
      fee_calc[i] = ecf[i] * variable_fee + fixed_fee
      nb[i] = (ecf[i] - fee_calc[i]) / pr[i]
      nbc[i] = nb[i] + nbc[i-1]
      val[i] = nbc[i] * pr[i]

    elif (sig[i] == 0) & (pos[i] == 0):       # do nothing, stay FLAT: add monthly invest to cash, add interest
      intr_calc[i] = cash[i-1] * inr[i] / 12
      cash[i] = cash[i-1] + intr_calc[i] + ecf[i]

    elif sig[i] == 1:                       # sell money fund & go LONG: add interest and monthly invest to cash, invest from cash into portfolio (2 x fees)
      intr_calc[i] = cash[i-1] * inr[i] / 12
      cash[i] = cash[i-1] + intr_calc[i] + ecf[i]
      fee_calc[i] = 2 * (cash[i] * variable_fee + fixed_fee)
      cash[i] = cash[i] - fee_calc[i]
      nb[i] = cash[i] / pr[i]
      nbc[i] = nb[i]
      val[i] = nbc[i] * pr[i]
      cash[i] = 0

    elif sig[i] == -1:                      # go FLAT and buy money fund: transfer from portfolio to money account - 2 x xfees
      nbc[i] = nbc[i-1]
      val[i] = nbc[i] * pr[i]
      fee_calc[i] = 2 * (val[i] * variable_fee + fixed_fee)
      cash[i] = val[i] - fee_calc[i] + ecf[i]
      nb[i], nbc[i], val[i] = 0, 0, 0

  result = pd.DataFrame({'extCF':ecf,
                         'price': pr,
                         'position': pos,
                         'signal': sig,
                         'number': nb,
                         'cum_nb': nbc,
                         'value': val,
                         'cash': cash,
                         'fee': fee_calc,
                         'intr': intr_calc},
                        index=prices_m.index)
  result['total'] = result['value'] + result['cash']

  return result

In [ ]:
# generation of trades and and calculating results
result1 = equity_trend_filter(prices_m, invest_1, position, signal, interest, fixed_fee, variable_fee)
result2 = equity_trend_filter(prices_m, invest_2, position, signal, interest, fixed_fee, variable_fee)

In [ ]:
# plotting equity curves
text_subtitles = [f'Equity Curve from {start} to {end}', 'Positions', 'Trading Signals', 'Money Fund', 'Portfolio-Value']
fig = make_subplots(rows=5, cols=1, subplot_titles = (text_subtitles), row_heights=[0.6, 0.1, 0.1, 0.1, 0.1],
                    shared_xaxes=True, vertical_spacing=0.075 )
fig.add_trace(go.Scatter(x=result1.index, y=result1.total, name='equity const. mtl. invest', line_color='lightskyblue'), row=1, col=1)
fig.add_trace(go.Scatter(x=result2.index, y=result2.total, name='equity adjuste mtl. invest', line_color='green'), row=1, col=1)
fig.add_trace(go.Scatter(x=position.index, y=position, name='position', line_color ='red', fill='tozeroy'), row=2, col=1)
fig.add_trace(go.Scatter(x=signal.index, y=signal, name='signal'), row=3, col=1)
fig.add_trace(go.Bar(x=result1.index, y=result1.cash, name='cash'), row=4, col=1)
fig.add_trace(go.Bar(x=result1.index, y=result1.value, name='interest', marker_color='indianred'), row=5, col=1)
rangeslider=False
fig.update_layout(template='plotly_dark', autosize=False, width=1200, height=1000, xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
# printing of results:

for i in range(2):
  if i == 0:
    result = result1
    text = 'Method: constant monthly investment using filter from EMA-cross-over-system ******'
  elif i == 1:
    result = result2
    text = 'Method: CPI-adapted monthly investment using filter from EMA-cross-over-system ******'
  total_invest = result['extCF'].sum()
  total_fees = result.fee.sum()
  total_interest = result.intr.sum()
  final_equity = result.total.iloc[-1]
  growth_factor = final_equity / total_invest
  print('****************************************************')
  print(text)
  print(f'invested ammount (total): {total_invest:,.2f} EURO           final equity:  {final_equity:,.2f}  EURO')
  print(f'growth-factor:            {growth_factor:.2f}                     fees:          {total_fees:,.2f}  EURO')
  print(f'interests:                {total_interest:,.2f}     ')
  print(' ')

****************************************************
Method: constant monthly investment using filter from EMA-cross-over-system ******
invested ammount (total): 37,700.00 EURO           final equity:  117,497.29  EURO
growth-factor:            3.12                     fees:          10,180.90  EURO
interests:                31,172.71     
 
****************************************************
Method: CPI-adapted monthly investment using filter from EMA-cross-over-system ******
invested ammount (total): 49,127.40 EURO           final equity:  137,339.13  EURO
growth-factor:            2.80                     fees:          11,323.75  EURO
interests:                34,956.33     
 
